Alrighty, let's crack some encryption.
Here, I've copied the ciphertexts from the problem statement and represented them as an array of ASCII values.

In [434]:
import numpy as np
import re

alphabet = set(range(ord('a'), ord('z')+1)) | set(range(ord('A'), ord('Z')+1)) \
         | set(map(ord, '.,?! :/')) | set(range(ord('1'), ord('9')+1))
ok = lambda x: x in alphabet
ok = np.vectorize(ok)

C = [
    'BB3A65F6F0034FA957F6A767699CE7FABA855AFB4F2B520AEAD612944A801E',
    'BA7F24F2A35357A05CB8A16762C5A6AAAC924AE6447F0608A3D11388569A1E',
    'A67261BBB30651BA5CF6BA297ED0E7B4E9894AA95E300247F0C0028F409A1E',
    'A57261F5F0004BA74CF4AA2979D9A6B7AC854DA95E305203EC8515954C9D0F',
    'BB3A70F3B91D48E84DF0AB702ECFEEB5BC8C5DA94C301E0BECD241954C831E',
    'A6726DE8F01A50E849EDBC6C7C9CF2B2A88E19FD423E0647ECCB04DD4C9D1E',
    'BC7570BBBF1D46E85AF9AA6C7A9CEFA9E9825CFD5E3A0047F7CD009305A71E'
]

C = np.array([np.array([int(x, 16) for x in re.findall('..', c)], 
               dtype=np.uint8) for c in C])

This function returns whether or not the first two digits of two byte in a binary representation are equal.
This will tell us whether the underlying plaintext characters are of the same 'type' (letters or spaces/caps).

In [435]:
def similar(x, y):
    return f'{x:08b}'[:2] == f'{y:08b}'[:2]

Now, as we've seen in the lecture, when some plaintexts contain spaces at a position, we can obtain the key by XOR'ing the corresponding ciphertext with the ASCII value of a space character.
To retrace which positions contained spaces, we divide the ciphertext characters into two groups that all have the same type.
Then, if one of the groups contains spaces, the ciphertext characters in there should all be exactly equal.

In [436]:
def crack_OTP(chars): 
    m = len(chars)
    
    # Either all spaces/caps or all letters here
    A = {i for i in range(m) if similar(chars[i], chars[0])}
        
    # Check if group characters are all equal
    if len({chars[i] for i in A}) != 1:
        # Take other group if not
        A = set(range(m)) - A
        
    # If A is empty, there are no spaces
    if not A:
        return 0
            
    space = A.pop()             
    return chars[space] ^ ord(' ')

In [437]:
key = np.array([crack_OTP(C[:, i]) for i in range(len(C[0]))])

def print_plain(C, k):
    for i, c in enumerate(C):
        print(i, repr(''.join(map(chr, c ^ k[:len(c)]))))
        
print_plain(C, key)

0 '» am pOaWn§ng a s\x85crOt missio\x80\x1e'
1 'ºe is Wh\\ ¡nly pe\x92soD to trus\x9a\x1e'
2 '¦he cuQr\\nº plan \x89s ^op secre\x9a\x1e'
3 '¥hen sKoLlª we me\x85t ^o do thi\x9d\x0f'
4 '» thinH Mh«y shou\x8cd Lollow hi\x83\x1e'
5 '¦his iP Iu¼er tha\x8e tBat one i\x9d\x1e'
6 '¼ot onF Zaªet is \x82et^er than §\x1e'


Obviously, this only works at a position if we have a space there in some ciphertext and no capitals
Let's use our knowledge of English to do this; looking at the first message, we can be pretty sure that the message should be "I am planning a secret mission.", so let's retrace the key from that and see whether we're correct:

In [438]:
m = "I am planning a secret mission."
key = [C[0][i] ^ ord(m[i]) for i in range(len(C[0]))]
print_plain(C, key)

0 'I am planning a secret mission.'
1 'He is the only person to trust.'
2 'The current plan is top secret.'
3 'When should we meet to do this?'
4 'I think they should follow him.'
5 'This is purer than that one is.'
6 'Not one cadet is better than I.'


In [447]:
C = [
    'f59fcab66630376f9958826a2b78ab35e347ab8f8674166bb7c4c6c1eb8644622fa20427e3602505d6b73f7eade19634387a8b3f76e2d5078a7f30',
    'ee91cfa97423362ad64acd3e207cfb02fe54a1dcd0580d63b491dbcab9c8667a33af4a2eee362549c0a03e73abead27d3f32877176e8d0068f7b6ccec5',
    'f6b381957039292ad05fcd3e207cfb28f843abdc8472056fb081cb8fb887436231ac1823af35330cd7f43f78e2fdd33e2428867160ebd3118c7371c5cb2ded268064',
    'f298c9e2703c2a7fd758cd252e39ad34fa5babdd91790b62ad90c6cab8c8537735b9063faf25380ad6b12f72a6afc2352e7a872975e2d506996e77c4853aa2',
    'f298c9b17471247ecd4d8e213b39b22ce654addbd06b176ca88dcc8fbf9a506532ed0328af34280c93b02e7aadecc43c3f33817175f5d9119d696d85',
    'ef848cab62712b65cd0c99222d39a928f15dba8f9d720e62a18ac1c6be85056229ed0321e12f320c939d1f3ab1ead5283933962825f7c41d9a767bc69867',
    'f682c3b674323163cf49cd272d78a834e450bd8f987a146be486cacaa5c8446027a40627ed2c254593a3236ee2e1d9296b2e8d7170f4d3528c727bc6d4',
    'f495cda6313029669958852f687dbe35f75ca2dcd07a162eac90dbdfb8d20a3931ba1d68ec232347d7b16472aca0c32d2f3b963476a88442c92d31db8864fb338926'
]

C = np.array([np.array([int(x, 16) for x in re.findall('..', c)], 
               dtype=np.uint8) for c in C])

In [448]:
n = len(max(C, key=len))
key = np.zeros(n, dtype=np.uint8)

In [449]:
for i in range(n):
    key[i] = crack_OTP([C[j][i] for j in range(len(C)) if len(C[j]) > i])

In [450]:
print_plain(C, key)

0 'õ\x9fktwa7e to cap5ãG« vt\x16es Æn na{/onol %lec{don is \x8bnsjqucr|'
1 'î\x91nker6 of the \x02þT¡s X\rmpuÛer Cc3b fav% stziied th\x87 s`ttfv k`'
2 'ö³ Wah) is the (øC«str\x05ateË sof{1ark u3ed {b recor\x86 ecwce~=`njo&\x80 '
3 'ò\x98h am*unt of v4ú[«ray\x0blitÆes vn5tlw e8ceekhd the \x87xpjqtpc;a } '
4 'ò\x98hse $ttacks i,æT\xadt k\x17bliÌ tru|2 i` t(e dj`ocrati\x81 p}}ctd! '
5 'ï\x84-is +ot the r(ñ]º mr\x0elenÁium {) iino2e I[ securi\x96y \x7f`os{7c= '
6 "ö\x82btec1ive meas4äP½ hz\x14e bÊen ay'ilobl%, wgt not t\x8d u|w e\x7f7cq"
7 'ô\x95ld a)l the de5÷\\¢s z\x16 htÛps:/ 1ww cc#.de hn/upda\x96es  0  }~-#y3\x89b'


Oei... That's not great.
However, by guessing (using our knowledge of English) prefixes of messages, we can obtain the key piece by piece.
Let's write a function that lets us do this:

In [451]:
def process(prefix, i):
    for j, s in enumerate(prefix):
        key[j] = ord(s) ^ C[i][j]

Message 3 starts with a three letter word and then "amount of", so it's likely that the message starts with "The amount of ".

In [452]:
process('The amount of ', 3)
print_plain(C, key)

0 'Software to cap5ãG« vt\x16es Æn na{/onol %lec{don is \x8bnsjqucr|'
1 'Hackers of the \x02þT¡s X\rmpuÛer Cc3b fav% stziied th\x87 s`ttfv k`'
2 'PC-Wahl is the (øC«str\x05ateË sof{1ark u3ed {b recor\x86 ecwce~=`njo&\x80 '
3 'The amount of v4ú[«ray\x0blitÆes vn5tlw e8ceekhd the \x87xpjqtpc;a } '
4 'These attacks i,æT\xadt k\x17bliÌ tru|2 i` t(e dj`ocrati\x81 p}}ctd! '
5 'It is not the r(ñ]º mr\x0elenÁium {) iino2e I[ securi\x96y \x7f`os{7c= '
6 "Protective meas4äP½ hz\x14e bÊen ay'ilobl%, wgt not t\x8d u|w e\x7f7cq"
7 'Read all the de5÷\\¢s z\x16 htÛps:/ 1ww cc#.de hn/upda\x96es  0  }~-#y3\x89b'


Let's repeat this every time we can guess a prefix:

In [453]:
process('Protective measures ', 6)
print_plain(C, key)

0 'Software to capture vt\x16es Æn na{/onol %lec{don is \x8bnsjqucr|'
1 'Hackers of the Chaos X\rmpuÛer Cc3b fav% stziied th\x87 s`ttfv k`'
2 'PC-Wahl is the investr\x05ateË sof{1ark u3ed {b recor\x86 ecwce~=`njo&\x80 '
3 'The amount of vulneray\x0blitÆes vn5tlw e8ceekhd the \x87xpjqtpc;a } '
4 'These attacks impact k\x17bliÌ tru|2 i` t(e dj`ocrati\x81 p}}ctd! '
5 'It is not the right mr\x0elenÁium {) iino2e I[ securi\x96y \x7f`os{7c= '
6 "Protective measures hz\x14e bÊen ay'ilobl%, wgt not t\x8d u|w e\x7f7cq"
7 'Read all the details z\x16 htÛps:/ 1ww cc#.de hn/upda\x96es  0  }~-#y3\x89b'


In [454]:
process('The amount of vulnerabilities ', 3)
print_plain(C, key)

0 'Software to capture votes in na{/onol %lec{don is \x8bnsjqucr|'
1 'Hackers of the Chaos Computer Cc3b fav% stziied th\x87 s`ttfv k`'
2 'PC-Wahl is the investigated sof{1ark u3ed {b recor\x86 ecwce~=`njo&\x80 '
3 'The amount of vulnerabilities vn5tlw e8ceekhd the \x87xpjqtpc;a } '
4 'These attacks impact public tru|2 i` t(e dj`ocrati\x81 p}}ctd! '
5 'It is not the right millennium {) iino2e I[ securi\x96y \x7f`os{7c= '
6 "Protective measures have been ay'ilobl%, wgt not t\x8d u|w e\x7f7cq"
7 'Read all the details at https:/ 1ww cc#.de hn/upda\x96es  0  }~-#y3\x89b'


In [455]:
process('PC-Wahl is the investigated software used ', 2)
print_plain(C, key)

0 'Software to capture votes in national elec{don is \x8bnsjqucr|'
1 'Hackers of the Chaos Computer Club have stziied th\x87 s`ttfv k`'
2 'PC-Wahl is the investigated software used {b recor\x86 ecwce~=`njo&\x80 '
3 'The amount of vulnerabilities vastly exceekhd the \x87xpjqtpc;a } '
4 'These attacks impact public trust in the dj`ocrati\x81 p}}ctd! '
5 'It is not the right millennium to ignore I[ securi\x96y \x7f`os{7c= '
6 'Protective measures have been available, wgt not t\x8d u|w e\x7f7cq'
7 'Read all the details at https://www.ccc.de hn/upda\x96es  0  }~-#y3\x89b'


In [459]:
process('The amount of vulnerabilities vastly exceeded the expectations.', 3)
print_plain(C, key)

0 'Software to capture votes in national election is insecure.'
1 'Hackers of the Chaos Computer Club have studied the software.'
2 'PC-Wahl is the investigated software used to record election da&\x80 '
3 'The amount of vulnerabilities vastly exceeded the expectations.'
4 'These attacks impact public trust in the democratic process.'
5 'It is not the right millennium to ignore IT-security problems.'
6 'Protective measures have been available, why not to use them?'
7 'Read all the details at https://www.ccc.de/en/updates/2017/pc-w3\x89b'


In [460]:
process('PC-Wahl is the investigated software used to record election data.', 2)
print_plain(C, key)

0 'Software to capture votes in national election is insecure.'
1 'Hackers of the Chaos Computer Club have studied the software.'
2 'PC-Wahl is the investigated software used to record election data.'
3 'The amount of vulnerabilities vastly exceeded the expectations.'
4 'These attacks impact public trust in the democratic process.'
5 'It is not the right millennium to ignore IT-security problems.'
6 'Protective measures have been available, why not to use them?'
7 'Read all the details at https://www.ccc.de/en/updates/2017/pc-wahl'


Success :)